# Async Loading Container

> Pattern for asynchronous content loading with skeleton loaders and loading indicators

In [ ]:
#| default_exp patterns.async_loading

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Any, Union
from enum import Enum
from fasthtml.common import *

from cjm_fasthtml_daisyui.components.feedback.loading import loading, loading_styles, loading_sizes
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, items, justify
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.core.base import combine_classes

## Loading Indicator Types

The `LoadingType` enum defines the types of loading indicators available.

In [ ]:
#| export
class LoadingType(Enum):
    """Types of loading indicators for async content."""
    SPINNER = "spinner"  # Default spinning indicator
    DOTS = "dots"  # Animated dots
    RING = "ring"  # Ring spinner
    BALL = "ball"  # Bouncing ball
    BARS = "bars"  # Loading bars
    INFINITY = "infinity"  # Infinity symbol
    NONE = "none"  # No loading indicator (for custom skeleton)

## AsyncLoadingContainer Function

The `AsyncLoadingContainer` function creates a container that displays a loading indicator and asynchronously loads content from a URL using HTMX.

Key features:
- Displays loading indicator initially
- Loads content via HTMX when triggered
- Replaces itself with loaded content (outerHTML swap)
- Customizable loading indicators
- Support for skeleton loaders
- Optional custom styling

In [ ]:
#| export
def AsyncLoadingContainer(
    container_id: str,  # HTML ID for the container
    load_url: str,  # URL to fetch content from
    loading_type: LoadingType = LoadingType.SPINNER,  # Type of loading indicator
    loading_size: str = "lg",  # Size of loading indicator (xs, sm, md, lg)
    loading_message: Optional[str] = None,  # Optional message to display while loading
    skeleton_content: Optional[Any] = None,  # Optional skeleton/placeholder content
    trigger: str = "load",  # HTMX trigger event (default: load on page load)
    swap: str = "outerHTML",  # HTMX swap method (default: replace entire container)
    container_cls: Optional[str] = None,  # Additional CSS classes for container
    **kwargs  # Additional attributes for the container
) -> FT:  # Div element with async loading configured
    """Create a container that asynchronously loads content from a URL."""
    # Build content based on loading type
    content_parts = []
    
    # Add skeleton content if provided
    if skeleton_content:
        content_parts.append(skeleton_content)
    elif loading_type != LoadingType.NONE:
        # Create loading indicator
        loading_indicator_parts = []
        
        # Map loading type to style
        style_map = {
            LoadingType.SPINNER: loading_styles.spinner,
            LoadingType.DOTS: loading_styles.dots,
            LoadingType.RING: loading_styles.ring,
            LoadingType.BALL: loading_styles.ball,
            LoadingType.BARS: loading_styles.bars,
            LoadingType.INFINITY: loading_styles.infinity,
        }
        
        # Map size string to size class
        size_map = {
            "xs": loading_sizes.xs,
            "sm": loading_sizes.sm,
            "md": loading_sizes.md,
            "lg": loading_sizes.lg,
        }
        
        style = style_map.get(loading_type, loading_styles.spinner)
        size = size_map.get(loading_size, loading_sizes.lg)
        
        # Add spinner
        loading_indicator_parts.append(
            Span(cls=combine_classes(loading, style, size))
        )
        
        # Add loading message if provided
        if loading_message:
            loading_indicator_parts.append(
                P(loading_message, cls=str(m.t(4)))
            )
        
        # Wrap in centered flex container
        content_parts.append(
            Div(
                *loading_indicator_parts,
                cls=combine_classes(
                    flex_display,
                    items.center,
                    justify.center,
                    p(4)
                )
            )
        )
    
    # Build container classes
    container_classes = []
    if container_cls:
        container_classes.append(container_cls)
    
    # Create the async loading container
    return Div(
        *content_parts,
        id=container_id,
        hx_get=load_url,
        hx_trigger=trigger,
        hx_swap=swap,
        cls=combine_classes(*container_classes) if container_classes else None,
        **kwargs
    )

## Usage Examples

Here are complete examples showing different use cases:

In [ ]:
from cjm_fasthtml_daisyui.components.data_display.card import card
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui

# Example 1: Simple spinner loader for a dashboard card
simple_loader = AsyncLoadingContainer(
    container_id="stats-card",
    load_url="/api/dashboard/stats",
    loading_message="Loading statistics...",
    container_cls=str(combine_classes(card, bg_dui.base_100))
)

print("Example 1: Simple spinner loader")
print(simple_loader)

Example 1: Simple spinner loader
stats-card


In [ ]:
# Example 2: Skeleton loader for card content
from cjm_fasthtml_daisyui.components.feedback.skeleton import skeleton

skeleton_card = Div(
    Div(cls=combine_classes(skeleton, "h-32", "w-full", m.b(4))),
    Div(cls=combine_classes(skeleton, "h-4", "w-full", m.b(2))),
    Div(cls=combine_classes(skeleton, "h-4", "w-3/4")),
    cls=str(p(4))
)

skeleton_loader = AsyncLoadingContainer(
    container_id="user-profile",
    load_url="/api/user/profile",
    loading_type=LoadingType.NONE,
    skeleton_content=skeleton_card,
    container_cls=str(combine_classes(card, bg_dui.base_100))
)

print("Example 2: Skeleton loader")
print(skeleton_loader)

Example 2: Skeleton loader
user-profile


In [ ]:
# Example 3: Different loading indicator styles
print("Example 3: Different loading styles")

for loading_style in [LoadingType.SPINNER, LoadingType.DOTS, LoadingType.RING]:
    loader = AsyncLoadingContainer(
        container_id=f"content-{loading_style.value}",
        load_url="/api/content",
        loading_type=loading_style,
        loading_size="md"
    )
    print(f"  {loading_style.value}: Created")

Example 3: Different loading styles
  spinner: Created
  dots: Created
  ring: Created


In [ ]:
# Example 4: Lazy loading with intersection observer
lazy_loader = AsyncLoadingContainer(
    container_id="lazy-content",
    load_url="/api/heavy-content",
    trigger="intersect once",  # Load only when scrolled into view
    loading_type=LoadingType.SPINNER,
    loading_message="Loading when visible..."
)

print("Example 4: Lazy loading")
print(lazy_loader)

Example 4: Lazy loading
lazy-content


## Server-Side Response

The endpoint specified in `load_url` should return the complete content that will replace the loading container. Since the default swap is `outerHTML`, the returned content should include the container with the same ID:

```python
@app.get("/api/dashboard/stats")
def get_stats():
    # Load your data
    stats = get_statistics()
    
    # Return complete card with same ID
    return Div(
        H3("Statistics"),
        P(f"Total: {stats['total']}"),
        P(f"Active: {stats['active']}"),
        id="stats-card",  # Same ID as the loading container
        cls=combine_classes(card, card_body, bg_dui.base_100)
    )
```

Alternatively, use `swap="innerHTML"` to replace only the container's contents:

```python
# Container
AsyncLoadingContainer(
    container_id="stats-card",
    load_url="/api/dashboard/stats",
    swap="innerHTML",
    container_cls=str(combine_classes(card, card_body, bg_dui.base_100))
)

# Endpoint - no need to include container
@app.get("/api/dashboard/stats")
def get_stats():
    stats = get_statistics()
    return Div(
        H3("Statistics"),
        P(f"Total: {stats['total']}"),
        P(f"Active: {stats['active']}")
    )
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()